### Gauss-Hermite & GPU model test

We begin by integrating the custom Gauss-Hermite integration step into the standard "serial" code:

In [58]:
#using Plots; gr()
using FastGaussQuadrature
include("../julia/aguirregabiria_core.jl")

compute_fixed_point (generic function with 1 method)

Notes:

1. GPU kernel will **not** perform interpolation
2. Integration with Gauss-Hermite is inlined

In [33]:
dmd_tf(x,y,z) = 1/sqrt(2*pi*sigma_eps^2)*exp.(-(x-(alpha+betas_transition.*log(y))).^2./(2*sigma_eps^2))

dmd_tf (generic function with 1 method)

In [34]:
@time dmd_transition_fs(1, 2, 1)

  0.071662 seconds (40.22 k allocations: 2.219 MiB)


3-element Array{Float64,1}:
 1.67834e-7
 0.0496474 
 0.199988  

In [35]:
@time dmd_tf(1, 2, 1)

  0.065473 seconds (11.48 k allocations: 631.941 KiB, 10.89% gc time)


3-element Array{Float64,1}:
 1.67834e-7
 0.0496474 
 0.199988  

In [36]:
lambdas      = generate_simplex_3dims(5)
l_price_grid = 10
min_price    = 0.5
max_price    = 1.5
price_grid   = linspace(min_price, max_price, l_price_grid)

function V_0(x) 
    optimal_price = myopic_price(x)
    return period_return(optimal_price, x)
end

V_0 (generic function with 1 method)

In [37]:
function rescale_demand(d, beta_l, price)
    # Rescales demand to use Gauss-Hermite 
    # collocation points.
    mu = alpha + beta_l*log.(price)
    
    return (sqrt(2)*sigma_eps*d + mu)
end

function adv_bellman_operator(Vguess, price_grid, lambda_simplex)
    
    policy = zeros(size(lambda_simplex, 1))
    T_V    = zeros(size(lambda_simplex, 1))

    # 1. Go over grid of state space
    # 2. Write objective (present return + delta*eOfV)
    # 3. Find optimal p on that objective
    # 4. Write optimal p and value function on that point in the grid
    
    GHx, GHw = gausshermite(20)
    
    for index in 1:size(lambda_simplex, 1)
        
        lw = lambda_simplex[index, :]
        
        # Compute period returns
        R = period_return(price_grid, lw)
        
        # Compute the expectation integral of V0
        E0fV_p = zeros(price_grid)
        
        for (i, price) in enumerate(price_grid)
            # Gauss-Hermite integration using pre-computed
            # collocations and weights.
            sum_over_each_lambda = 0.0
            for (l, beta_l) in enumerate(betas_transition)
                for (k, hermite_point) in enumerate(GHx)
                    rescaled_remand = rescale_demand(hermite_point, beta_l, price)
                    new_lw          = update_lambdas(rescaled_remand, price, ~, lw)
                    #v_value = f(new_lambdas_val)
                    sum_over_each_lambda += Vguess(new_lw)*GHw[k]*lw[l]
            
                end
            end
            E0fV_p[i] = (1/sqrt(pi) * sum_over_each_lambda)
        end
        
        #println(E0fV_p)
            
        # Maximize value function
        objective_vals = R + (delta *  E0fV_p)
            
        T_V[index], ind_max = findmax(objective_vals)                       
        policy[index] = price_grid[ind_max]

    end

    # Interpolate
    interp_T_V = interpV(lambda_simplex, T_V)

    return interp_T_V, policy
    
end

@time adv_bellman_operator(V_0, price_grid, lambdas)

  0.290143 seconds (558.10 k allocations: 19.583 MiB, 2.56% gc time)


(interpolate_V, [1.5, 1.5, 1.16667, 0.722222, 0.722222, 0.722222])

In [38]:
@time bellman_operator(V_0, price_grid, lambdas)

  0.730981 seconds (1.89 M allocations: 66.456 MiB, 2.86% gc time)


(interpolate_V, [1.5, 1.5, 1.16667, 0.722222, 0.722222, 0.722222])

In [61]:
function adv_compute_fixed_point(V, price_grid, lambda_simplex; error_tol=1e-5, max_iter=50, verbose=true, skip=10)

    iterate = 1
    error = error_tol + 1
    
    policy = similar(price_grid)
    error  = Inf

    while iterate <= max_iter && error > error_tol
        if verbose && (mod(iterate, skip) == 0)
            tic()
        end

        new_V, policy = adv_bellman_operator(V, price_grid, lambda_simplex)
                        
        # Compute error over simplex:
        error = 0
        for i in 1:size(lambda_simplex,1)
            error += abs(V(lambda_simplex[i,:])-new_V(lambda_simplex[i,:]))
        end

        V = new_V

        if verbose && (mod(iterate, skip) == 0)
            println(@sprintf("Computed iterate %d with error %.4f", iterate, error))
            print(" !-- ")
            toc()
        end
                        
        iterate += 1                
                        
    end

    return V, policy, error

end

@time adv_compute_fixed_point(V_0, price_grid, lambdas)

Computed iterate 10 with error 4.1581
 !-- elapsed time: 0.063619901 seconds
Computed iterate 20 with error 1.4496
 !-- elapsed time: 0.10724422 seconds
Computed iterate 30 with error 0.5054
 !-- elapsed time: 0.062294531 seconds
Computed iterate 40 with error 0.1762
 !-- elapsed time: 0.063354718 seconds
Computed iterate 50 with error 0.0614
 !-- elapsed time: 0.064594215 seconds
  3.645378 seconds (23.26 M allocations: 1.059 GiB, 8.06% gc time)


(interpolate_V, [1.5, 1.5, 1.16667, 0.722222, 0.722222, 0.722222], 0.06144856657666509)

In [62]:
@time compute_fixed_point(V_0, price_grid, lambdas)

Computed iterate 10 with error 4.1581
 !-- elapsed time: 0.308997445 seconds
Computed iterate 20 with error 1.4496
 !-- elapsed time: 0.281893798 seconds
Computed iterate 30 with error 0.5054
 !-- elapsed time: 0.282547842 seconds
Computed iterate 40 with error 0.1762
 !-- elapsed time: 0.273341006 seconds
Computed iterate 50 with error 0.0614
 !-- elapsed time: 0.270560107 seconds
 14.236414 seconds (108.40 M allocations: 4.833 GiB, 8.24% gc time)


(interpolate_V, [1.5, 1.5, 1.16667, 0.722222, 0.722222, 0.722222], 0.061448522714727005)

#### GPUArrays extension

In [9]:
using GPUArrays, CLArrays

gpu_A = ones(CLArray{Float32}, 10, 1)

gpu_call(gpu_A, (gpu_A,)) do state, gpu_A
  begin
     
        function _add(a::Float32, b::Float32)::Float32
            return (a + b)*2
        end
        
        i = @linearidx gpu_A;
        
        gpu_A[i] = _add(1.0, 2.0);        
  end
end

___________________________________________________________________
Error in Expr rewrite! AssertionError("Found non concrete return type: Any")

Expression resulting in the error: 
(_add::#_add#16)(1.0, 2.0)::Any
happening in function tree:
Sugar stack trace:
  [1] #14(CLArrays.KernelState, CLArrays.DeviceArray{Float32,2,Transpiler.CLIntrinsics.GlobalPointer{Float32}})


Code of the context this error occured in: 
begin 
      NewvarNode(:(i::UInt32))
      _add::#_add#16 = $(Expr(:new, :(Main.#_add#16)))
      x1::CLArrays.DeviceArray{Float32,2,Transpiler.CLIntrinsics.GlobalPointer{Float32}} = gpu_A::CLArrays.DeviceArray{Float32,2,Transpiler.CLIntrinsics.GlobalPointer{Float32}}
      i1::UInt32 = (GPUArrays.linear_index)(state::CLArrays.KernelState)::UInt32
      if (i1::UInt32 > (GPUArrays.length)(x1::CLArrays.DeviceArray{Float32,2,Transpiler.CLIntrinsics.GlobalPointer{Float32}})::UInt32)::Bool
          $(Expr(:return))
      end::Any
      i::UInt32 = i1::UInt32
      SSAValue(0) 

LoadError: [91mAssertionError: Found non concrete return type: Any[39m